In [1]:
# Libraries
import pandas as pd
pd.set_option('display.max_columns', 40)
pd.set_option('display.width', 2000)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Remove printing error
pd.options.mode.chained_assignment = None

In [51]:
# Import
path = r'switch_data/antibiotic_po_flag.csv'
antibiotic_po_flag = pd.read_csv(path)

In [52]:
import json, snowflake.connector

# establish the connection to snowflake
ctx = snowflake.connector.connect( 
    **json.load(open('/opt/ich/python-snowflake-defaults.json')))
    
# verify and test if connection is working
try: 
    cs = ctx.cursor() 
    cs.execute('SELECT current_version(), current_role(), current_warehouse()')
    print(cs.fetchone())
finally: 
    cs.close()

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/snowflake/connector/options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (6.0.0), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
We were unable to open a browser window for you, please open the following url manually then paste the URL you are redirected to into the terminal.
URL: https://login.microsoftonline.com/be00e2c6-806c-45f8-84c3-3fc99f64b8d3/saml2?SAMLRequest=pZNPc%2BIgGIe%2FSoY9B8ifasIYO7a2s87Y1tW4290bEmIZE0iBGOunX4x1pntoL3tj4Hnh4f3B6PpQV96eayOUzEAAMfC4ZKoQcpuBdX7vJ8AzlsqCVkryDLxxA67HI0PrqiGT1r7IJX9tubGe20ga0i9koNWSKGqEIZLW3BDLyGryMCchxKTRyiqmKvCh5OsKagzX1hleSgojnN6LtQ1BqOs62EVQ6S0KMcYIp8hRJ%2BTbhT%2B4O33CBwjHJ94RDl%2B8u90IeW7BV1qbM2TI9zxf%2BIunVQ68yUX1VknT1lyvuN4LxtfL%2BVnAOANlo2B4hWG7841yPYSNFntqeSXkDhqpurKiO85U3bTWHQDdCJW8QJXaCteD2TQDzU4U7PfqlwzivQr0%2Fgd9fkjXNzYvo3T5fDzezY53cxxPDk%2FD13gXM%2BD9vIQcnkKeGdPymTxFa90UDiMfD%2FwgzXFK8JCECRwO4j%2FAm7pohaS2r7z49x6wFkwro0qrpBPn

('7.20.0', 'ICHT_RG_COVOAM_22016_PROD', 'ICHT_WH_COVOAM_22016_PROD')


True

In [53]:
# Import 
query = '''
SELECT * from ICHT_SANDBOX_PROD.COVOAM_22016.ANTI_INFECTIVE_ADMINISTRATION
'''
cur = ctx.cursor().execute(query)
drug_df = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])

In [54]:
# Import 
query = '''
SELECT * from ICHT_PROD.ICHT_COVID.EPISODES
'''
cur = ctx.cursor().execute(query)
episodes = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])

In [55]:
# Import
path = r'switch_data/anti_infectives_flag_final.csv'
anti_infectives_flag_final = pd.read_csv(path)

In [56]:
# Filter for relevant delivery methods
antibiotics = drug_df
antibiotics = antibiotics.groupby('ROUTE').filter(lambda x: len(x) > 3000)
antibiotics['ROUTE'] = antibiotics['ROUTE'].replace({'oral': 'PO', 'enteral':'PO', 'oromucosal':'PO', 'oral/NG':'PO', 'NG': 'PO'})
antibiotics['ADMINISTRATION_DATETIME'] = pd.to_datetime(antibiotics['ADMINISTRATION_DATETIME']).dt.date
# Get list of antibiotics
antibiotic_list = anti_infectives_flag_final[anti_infectives_flag_final['switch_antibiotic_flag'] == 'Yes']['MEDICATION_NAME_SHORT'].to_list()
len(antibiotic_list)
# Filter for antibiotics
antibiotics = antibiotics[antibiotics['MEDICATION_NAME_SHORT'].isin(antibiotic_list)]
# Filter for those with IV and PO
filtered_antibiotics = antibiotics[antibiotics['SUBJECT'].isin((antibiotics.groupby(['SUBJECT'])['ROUTE'].nunique() > 1).loc[lambda x : x == True].index.to_list())]
# Merge with episodes
antibiotic_episodes = pd.merge(filtered_antibiotics[['SUBJECT', 'MEDICATION_NAME_SHORT', 'ADMINISTRATION_DATETIME', 'ROUTE']], episodes[['SUBJECT', 'SPELL_IDENTIFIER']])

73

In [57]:
antibiotic_po_flag = antibiotic_po_flag[['SPELL_IDENTIFIER', 'ADMINISTRATION_DATETIME', 'ROUTE']]
antibiotic_po_flag_iv = antibiotic_po_flag[antibiotic_po_flag['ROUTE'] == 'IV']
antibiotic_po_flag_po = antibiotic_po_flag[antibiotic_po_flag['ROUTE'] == 'PO']
antibiotic_po_flag_iv.drop_duplicates(subset=['SPELL_IDENTIFIER', 'ROUTE'], keep='last', inplace=True)
antibiotic_po_flag_po.drop_duplicates(subset=['SPELL_IDENTIFIER', 'ROUTE'], keep='first', inplace=True)

# Convert all to string so merge works
antibiotic_episodes = antibiotic_episodes.astype(str)
antibiotic_po_flag_po = antibiotic_po_flag_po.astype(str)
antibiotic_po_flag_iv = antibiotic_po_flag_iv.astype(str)

antibiotic_po_flag_po2 = antibiotic_po_flag_po.merge(antibiotic_episodes.drop(columns=['SUBJECT']))
antibiotic_po_flag_po2.rename(columns={'MEDICATION_NAME_SHORT':'antibiotics'}, inplace=True)
antibiotic_po_flag_po2.drop_duplicates(inplace=True)
antibiotic_po_flag_po2 = antibiotic_po_flag_po2.groupby(['SPELL_IDENTIFIER', 'ADMINISTRATION_DATETIME', 'ROUTE'])['antibiotics'].apply(lambda x: ','.join(x)).reset_index()

antibiotic_po_flag_iv2 = antibiotic_po_flag_iv.merge(antibiotic_episodes.drop(columns=['SUBJECT']))
antibiotic_po_flag_iv2.rename(columns={'MEDICATION_NAME_SHORT':'antibiotics'}, inplace=True)
antibiotic_po_flag_iv2.drop_duplicates(inplace=True)
antibiotic_po_flag_iv2 = antibiotic_po_flag_iv2.groupby(['SPELL_IDENTIFIER', 'ADMINISTRATION_DATETIME', 'ROUTE'])['antibiotics'].apply(lambda x: ','.join(x)).reset_index()

antibiotics_df = pd.concat([antibiotic_po_flag_iv2 ,antibiotic_po_flag_po2], ignore_index = True)
antibiotics_df.sort_values(by=['SPELL_IDENTIFIER', 'ADMINISTRATION_DATETIME'], inplace=True)

In [80]:
# Save
#antibiotics_df.to_csv('switch_data/antibiotics.csv', index=False)